# Creating the SML URN Hash  
The SML look-up is a NAPTR DNS query which returns the URL of the SMP. The record queried is a URN constructed from the combined Specification, Schema, and Party ID and then hashed.  After the hash is created, the URN is combined with the look-up domain.  This combination of URN and domain is the record locator for the information in the DNS.  

The Python code, as script, function, or method, provides elaboration and implementation of the process as described in Section 3.2 of the [BPC SML Profile Version 1.0](https://github.com/BPC-Exchange-Framework/BPC-Market-Pilot/blob/f3844411c40dd4b84276fd3ef6020d247afd83c4/BPC%20SML%20Profile%20Version%201.0.pdf) document found on the [BPC-Exchange-Framework/BPC-Market-Pilot Github site](https://github.com/BPC-Exchange-Framework/BPC-Market-Pilot).  
<br/>

<div class="alert alert-block alert-info">
<b>&#10071;</b>The BPC SML Domain is: <b>bpcb2b.net</b>
</div>

<br/>
## Information needed to create the URN
Three data points are required to create the URN:  
 <ul>
   <li>Specification  (urn:oasis:names:tc:ebcore:partyid-type)</li>
    <li>Schema (iso6523:0060)  </li>
    <li>PartyID</li>
</ul>  
<br/>
<div class="alert alert-block alert-info"><p><b>&#10071;<i>Format of Specification, Schema, and Party ID is assumed to be conformant.</i></b></p>
<p>    The standard articulated for an <b>ebCorePartyId</b> referenced in the [Business Document Metadata Service Location Version 1.0](https://docs.oasis-open.org/bdxr/BDX-Location/v1.0/os/BDX-Location-v1.0-os.html) referring to the [OASIS ebCore Party Id Type Technical Specification Version 1.0](https://docs.oasis-open.org/ebcore/PartyIdType/v1.0/PartyIdType-1.0.odt) provides normative guidelines for the values used for the Specification and Schema.   </p>
    </div>
<br/>
    This code does not validate or enforce compliance with those standards. There is no Inspection or validation of values used for the Specification, Schema, and Party ID used to create a URN hash for look-up.  Conformance for those values are assumed, however the code works on any string of data. 
          
    
<br/>  


## Example - Creating the URN hash
<br/>  

### 1. Import Modules  
The ```hashlib``` and ```bas64``` Python modules are used in this process.  

    import hashlib
    import base64
<br/>    


### 2. Concatenate the string    
Strings are a primitive data type in Python.  Instantiate and initialize string variables for the 
individual data values and concatenate them into a single string.   Simple validation of the correct 
format of the urn is included.  

    specification = "urn:oasis:names:tc:ebcore:partyid-type:unregistered:myscheme"  
    schema = "BPC01"  
    party_id = "bpcBusid01"   
    urn = specification + ":" + schema + "::" + party_id  
    urn_test_case = 
        "urn:oasis:names:tc:ebcore:partyid-type:unregistered:myscheme:BPC01::bpcBusid01"
    print(f"urn is concatenated properly: {urn == urn_test_case}")  
    print(urn)


<div class="alert alert-block alert-warning"><p><b>&#10071;<i>Punctuation</i></b></p><p>
    Note the use of a single and a double colon as separators between the values.  These are 
    includes as part of the string and are <b>required.</b>  </p>
    </div>


<br/>  


### 3. Convert to lower-case  
Implements the Python String ```lower()``` method.

    lower_case_urn = urn.lower()
    print(f"URN converted to lower case: {lower_case_urn}")

<br/>  



### 4. Encode as utf-8  
The sha256 and base32 operations done on the string are Buffered Protocols, 
which requires that they be in a 
[bytes-like object](https://docs.python.org/3/glossary.html#term-bytes-like-object) 
format in Python.  This is obtained using the String ```encode()``` 
method specifying "utf-8" as the format.  
The ```bytes(x, encoding, error)``` function could also be used here. 
    
    urn_encoded =  lower_case_urn.encode('utf-8')
    print (f"URN encoded as utf-8: {urn_encoded}")

<br/>  


### 5. Create sha256 hash  
This implements the ```sha256()``` method of the hashlib module imported in Step \#1.     
    
    sha256_urn  = hashlib.sha256(urn_encoded)
    print (f"URN hashed using sha256 {sha256_urn}")  

<br/>    
  
### 6. Obtain the 'digest'   
The digest is the concatenation of all of the data fed into the hash so far, i.e., the current value.  
(Though implemented as a single operation here, the buffered protocol allows for additions and updates to the hash.)  The ```digest()``` method is included in the ```hashlib``` module.

    sha256_digest = sha256_urn.digest()
    print(f"Digest of buffered stream containing results thus far: {sha256_digest}")

<br/>  




### 7.  Encode into base32 
<div class="alert alert-block alert-warning"><p><b>&#10071;<i>Why encode in base32?</i></b></p><p>
    Encryption using sha256 results in a one-way hash. The original value of the URN is not intended to be derived from that hash as that is cryptographically impossible.  (Encoding the sha256 hash into base32 is __not__ one way.)  <br/><br/>The sha256 hash is 256 bits, or 32 bytes. A two digit hexadecimal representation of the 32 byte hash is 64 characters long.  Since base32 encoding has a character set of 32 compared to hexadecimal's 16, the sha256 hashed value can be represented in 32 characters in base32 instead of the 64 required for hex.  <br/><br/>
    The irreversibility of the sha256 hash implies that the process is cryptographically significant.  It is not.  The hashed URN is ultimately used as a dictionary look-up in the DNS record _for a given domain_. Where a common specification and schema are in use by many participants, it becomes imperative that the PartyIDs are unique _for a specific domain_.  Otherwise, while a sha256 hash can't be reversed, it can be __duplicated__.  <br/><br/>
    The output of this process is a 256 bit/32 byte value represented in a base 32 character set.
    </p>
    </div>
    

Take the digest entry, still a bytes-like object, and encode it in base32, 
resulting in a string 32 characters in length.  This implements the ```b32encode``` 
method of the base64 module.

    b32_urn = base64.b32encode(sha256_digest)
    print(f"The base32 encoded representation of the URN: {b32_urn}")

<br/>    

### 8.  Strip off extras  
The base32 encoding may result in extra characters at the end of the string.  
The ```rstrip``` String method is used to 
remove any of this additional padding at the end of the string.

    b32_urn_clean = b32_urn.rstrip(b"=")
    print(f"The URN with any padding removed {b32_urn_clean}")


<br/>    



### 9. Convert back to a String  
The object is still in a binary or bytes-like object format.  
Convert it back into a String primitive using the String ```decode('utf-8)``` method 
where 'utf-8' was the original encoding method.  

    b32_str = b32_urn_clean.decode('utf-8')

<br/>    



### 10. Convert to lower-case  
Per the specification, ensure the output is entirely in lowercase. 
This implements the String ```lower()``` method again.  

    final = b32_str.lower()

<br/>   


### Final Output  
The final output of the hash algorithm.

    print(f"The final result: {final}")
    
    


<hr>
<br/>  
<br/>  



## Code
The entirety of the Python code for the hash algorithm - can be run as a script or a function.

        #########################################################
        #
        #  Example Constructing a URN for SML DNS NAPTR look-up
        #
        ##########################################################
        # import the modules
        import hashlib
        import base64
        # get the urn
        specification = "urn:oasis:names:tc:ebcore:partyid-type:unregistered:myscheme"
        schema = "BPC01"
        party_id = "bpcBusid01"
        urn = specification + ":" + schema + "::" + party_id
        urn_test_case = 
            "urn:oasis:names:tc:ebcore:partyid-type:unregistered:myscheme:BPC01::bpcBusid01"
        print(f"urn is concatenated properly: {urn == urn_test_case}")
        print(urn)
        # make sure it's converted to lower case
        lower_case_urn = urn.lower()
        print(f"URN converted to lower case {lower_case_urn}")
        # has to be a byte-like object to be hashed, so encode it as utf-8
        urn_encoded =  lower_case_urn.encode('utf-8')
        print (f"URN encoded as utf-8: {urn_encoded}")
        # now create the sha256 hash of it
        sha256_urn  = hashlib.sha256(urn_encoded)
        # get the current value of the buffer stream
        sha256_digest = sha256_urn.digest()
        print(f"Digest of buffered stream containing results thus far: {sha256_digest}")
        #encode into b32 
        b32_urn = base64.b32encode(sha256_digest)
        print(f"The base32 encoded representation of the URN: {b32_urn}")
        # strip off the equals sign.... 
        b32_urn_clean = b32_urn.rstrip(b"=")
        print(f"The URN with any padding removed {b32_urn_clean}")
        # convert it back to string.
        b32_str = b32_urn_clean.decode('utf-8')
        # make sure it's in lower case again.
        final = b32_str.lower()
        # This should be your final answer
        print(f"The final result: {final}")
<hr>  
<br/>  



## Output 

    urn is concatenated properly: True
    urn:oasis:names:tc:ebcore:partyid-type:unregistered:myscheme:BPC01::bpcBusid01
    URN converted to lower case: urn:oasis:names:tc:ebcore:partyid-type:unregistered:myscheme:bpc01::bpcbusid01
    URN encoded as utf-8: b'urn:oasis:names:tc:ebcore:partyid-type:unregistered:myscheme:bpc01::bpcbusid01'
    URN hashed using sha256 <sha256 _hashlib.HASH object @ 0x106202710>
    Digest of buffered stream containing results thus far: b'\xc3{4\xfc3"\xdb\xc1u\xdcd\xe8\xbf\xe2\xad\x86\xdfjxob\x1e\'\x17\x8f\xb0\x83!\xec\x15\xab~'
    The base32 encoded representation of the URN: b'YN5TJ7BTELN4C5O4MTUL7YVNQ3PWU6DPMIPCOF4PWCBSD3AVVN7A===='
    The URN with any padding removed b'YN5TJ7BTELN4C5O4MTUL7YVNQ3PWU6DPMIPCOF4PWCBSD3AVVN7A'
    The final result: yn5tj7bteln4c5o4mtul7yvnq3pwu6dpmipcof4pwcbsd3avvn7a

<br/><br/>

In [31]:
#########################################################
#
#  Example Constructing a URN for SML DNS NAPTR look-up
#
##########################################################
# import the modules
import hashlib
import base64
# get the urn
specification = "urn:oasis:names:tc:ebcore:partyid-type:unregistered:myscheme"
schema = "BPC01"
party_id = "bpcBusid01"
urn = specification + ":" + schema + "::" + party_id
urn_test_case = "urn:oasis:names:tc:ebcore:partyid-type:unregistered:myscheme:BPC01::bpcBusid01"
print(f"urn is concatonated properly: {urn == urn_test_case}")
print(urn)
# make sure it's converted to lower case
lower_case_urn = urn.lower()
print(f"URN converted to lower case: {lower_case_urn}")
# has to be a byte-like object to be hashed, so encode it as utf-8
urn_encoded =  lower_case_urn.encode('utf-8')
print (f"URN encoded as utf-8: {urn_encoded}")
# now create the sha256 hash of it
sha256_urn  = hashlib.sha256(urn_encoded)
print (f"URN hashed using sha256 {sha256_urn}") 
# get the current value of the buffer stream
sha256_digest = sha256_urn.digest()
print(f"Digest of buffered stream containing results thus far: {sha256_digest}")
#encode into b32 
b32_urn = base64.b32encode(sha256_digest)
print(f"The base32 encoded representation of the URN: {b32_urn}")
# strip off the equals sign.... 
b32_urn_clean = b32_urn.rstrip(b"=")
print(f"The URN with any padding removed {b32_urn_clean}")
# convert it back to string.
b32_str = b32_urn_clean.decode('utf-8')
# make sure it's in lower case again.
final = b32_str.lower()
# This should be your final answer
print(f"The final result: {final}")

urn is concatonated properly: True
urn:oasis:names:tc:ebcore:partyid-type:unregistered:myscheme:BPC01::bpcBusid01
URN converted to lower case: urn:oasis:names:tc:ebcore:partyid-type:unregistered:myscheme:bpc01::bpcbusid01
URN encoded as utf-8: b'urn:oasis:names:tc:ebcore:partyid-type:unregistered:myscheme:bpc01::bpcbusid01'
URN hashed using sha256 <sha256 _hashlib.HASH object @ 0x106202710>
Digest of buffered stream containing results thus far: b'\xc3{4\xfc3"\xdb\xc1u\xdcd\xe8\xbf\xe2\xad\x86\xdfjxob\x1e\'\x17\x8f\xb0\x83!\xec\x15\xab~'
The base32 encoded representation of the URN: b'YN5TJ7BTELN4C5O4MTUL7YVNQ3PWU6DPMIPCOF4PWCBSD3AVVN7A===='
The URN with any padding removed b'YN5TJ7BTELN4C5O4MTUL7YVNQ3PWU6DPMIPCOF4PWCBSD3AVVN7A'
The final result: yn5tj7bteln4c5o4mtul7yvnq3pwu6dpmipcof4pwcbsd3avvn7a


In [2]:
#############################################
#
#  Test Case 3
#
#############################################
# import your libs
import hashlib
import base64
# get the urn
specification = "urn:oasis:names:tc:ebcore:partyid-type"
schema = "iso6523:0088"
party_id = "EAN-7638725972413"
urn = specification + ":" + schema + "::" + party_id
# urn = "urn:oasis:names:tc:ebcore:partyid-type:iso6523:0088::EAN-7638725972413"
print(urn)
# make sure it's converted to lower case
lower_case_urn = urn.lower()
print(lower_case_urn)
# has to be a byte-like object to be hashed, so encode it as utf-8
urn_encoded =  lower_case_urn.encode('utf-8')
print (urn_encoded)
# now create the sha256 hash of it
sha256_urn  = hashlib.sha256(urn_encoded)
# print(sha256_urn)  # this will be an object
# convert to human readable formats
sha256_digest = sha256_urn.digest()
print(sha256_digest)
#encode into b32 
b32_urn = base64.b32encode(sha256_digest)
print(b32_urn)
# strip off the equals sign.... 
b32_urn_clean = b32_urn.rstrip(b"=")
print(b32_urn_clean)
# convert it back to string in case you want to do anything with it.
b32_str = b32_urn_clean.decode('utf-8')
# make sure it's in lower case again.
final = b32_str.lower()
# This should be your final answer
print(final)

urn:oasis:names:tc:ebcore:partyid-type:iso6523:0088::EAN-7638725972413
urn:oasis:names:tc:ebcore:partyid-type:iso6523:0088::ean-7638725972413
b'urn:oasis:names:tc:ebcore:partyid-type:iso6523:0088::ean-7638725972413'
b'\x08\xb4\xd9\xaa\xf9\xe6\x80\xcb\xec\x1d\x8b\x12\x19\x00\x92\x7fh*\x0bh\x12\x1e\xe9({\x85\x9a\x9f\x07\x96I\xf5'
b'BC2NTKXZ42AMX3A5RMJBSAESP5UCUC3ICIPOSKD3QWNJ6B4WJH2Q===='
b'BC2NTKXZ42AMX3A5RMJBSAESP5UCUC3ICIPOSKD3QWNJ6B4WJH2Q'
bc2ntkxz42amx3a5rmjbsaesp5ucuc3iciposkd3qwnj6b4wjh2q


In [3]:
#############################################
#
#  Test Case 4
#
#############################################
# import your libs
import hashlib
import base64
# get the urn
specification = "urn:oasis:names:tc:ebcore:partyid-type"
schema = "iso6523:0088"
party_id = "bpc-2343030383"
urn = specification + ":" + schema + "::" + party_id
# urn = "urn:oasis:names:tc:ebcore:partyid-type:iso6523::bpc-2343030383"
print(urn)
# make sure it's converted to lower case
lower_case_urn = urn.lower()
print(lower_case_urn)
# has to be a byte-like object to be hashed, so encode it as utf-8
urn_encoded =  lower_case_urn.encode('utf-8')
print (urn_encoded)
# now create the sha256 hash of it
sha256_urn  = hashlib.sha256(urn_encoded)
# print(sha256_urn)  # this will be an object
# convert to human readable formats
sha256_digest = sha256_urn.digest()
print(sha256_digest)
#encode into b32 
b32_urn = base64.b32encode(sha256_digest)
print(b32_urn)
# strip off the equals sign.... 
b32_urn_clean = b32_urn.rstrip(b"=")
print(b32_urn_clean)
# convert it back to string in case you want to do anything with it.
b32_str = b32_urn_clean.decode('utf-8')
# make sure it's in lower case again.
final = b32_str.lower()
# This should be your final answer
print(final)

urn:oasis:names:tc:ebcore:partyid-type:iso6523:0088::bpc-2343030383
urn:oasis:names:tc:ebcore:partyid-type:iso6523:0088::bpc-2343030383
b'urn:oasis:names:tc:ebcore:partyid-type:iso6523:0088::bpc-2343030383'
b'\x8a.\xa1\x87T\xed\xee_\xd6h!3\n\x81W\xect\x11m\x146I\x01\xef\xa8\t\x065\xb8\x80y|'
b'RIXKDB2U5XXF7VTIEEZQVAKX5R2BC3IUGZEQD35IBEDDLOEAPF6A===='
b'RIXKDB2U5XXF7VTIEEZQVAKX5R2BC3IUGZEQD35IBEDDLOEAPF6A'
rixkdb2u5xxf7vtieezqvakx5r2bc3iugzeqd35ibeddloeapf6a


In [4]:
#############################################
#
#  Test Case 5
#
#############################################
# import your libs
import hashlib
import base64
# get the urn
specification = "urn:oasis:names:tc:ebcore:partyid-type"
schema = "iso6523:0088"
party_id = "4035811991021"
urn = specification + ":" + schema + "::" + party_id
# urn = "urn:oasis:names:tc:ebcore:partyid-type:iso6523:0088:4035811991021"
print(urn)
# make sure it's converted to lower case
lower_case_urn = urn.lower()
print(lower_case_urn)
# has to be a byte-like object to be hashed, so encode it as utf-8
urn_encoded =  lower_case_urn.encode('utf-8')
print (urn_encoded)
# now create the sha256 hash of it
sha256_urn  = hashlib.sha256(urn_encoded)
# print(sha256_urn)  # this will be an object
# convert to human readable formats
sha256_digest = sha256_urn.digest()
print(sha256_digest)
#encode into b32 
b32_urn = base64.b32encode(sha256_digest)
print(b32_urn)
# strip off the equals sign.... 
b32_urn_clean = b32_urn.rstrip(b"=")
print(b32_urn_clean)
# convert it back to string in case you want to do anything with it.
b32_str = b32_urn_clean.decode('utf-8')
# make sure it's in lower case again.
final = b32_str.lower()
# This should be your final answer
print(final)

urn:oasis:names:tc:ebcore:partyid-type:iso6523:0088::4035811991021
urn:oasis:names:tc:ebcore:partyid-type:iso6523:0088::4035811991021
b'urn:oasis:names:tc:ebcore:partyid-type:iso6523:0088::4035811991021'
b"H\xb9+'\x08y\xe2\xb5\xf0\xc5:\xe9\xa2\x0c\x87}\x96\xa6A\x92h\x85\xa5\x8brw\xd9\xd4\xd4\xc9Q,"
b'JC4SWJYIPHRLL4GFHLU2EDEHPWLKMQMSNCC2LC3SO7M5JVGJKEWA===='
b'JC4SWJYIPHRLL4GFHLU2EDEHPWLKMQMSNCC2LC3SO7M5JVGJKEWA'
jc4swjyiphrll4gfhlu2edehpwlkmqmsncc2lc3so7m5jvgjkewa


In [5]:
hashlib.sha256(b'urn:oasis:names:tc:ebcore:partyid-type:unregistered:myscheme:bpc01::bpcbusid01')

<sha256 _hashlib.HASH object @ 0x10551bed0>

In [6]:
hashlib.sha256(b'urn:oasis:names:tc:ebcore:partyid-type:unregistered:myscheme:bpc01::bpcbusid01').digest()

b'\xc3{4\xfc3"\xdb\xc1u\xdcd\xe8\xbf\xe2\xad\x86\xdfjxob\x1e\'\x17\x8f\xb0\x83!\xec\x15\xab~'

In [7]:
base64.b32encode(b'\xc3{4\xfc3"\xdb\xc1u\xdcd\xe8\xbf\xe2\xad\x86\xdfjxob\x1e\'\x17\x8f\xb0\x83!\xec\x15\xab~')

b'YN5TJ7BTELN4C5O4MTUL7YVNQ3PWU6DPMIPCOF4PWCBSD3AVVN7A===='

In [8]:
#!/usr/bin/env python3
#
# File: app_logging.py
# About: Logging provider
# Development: Kelly Kinney, Leo Rubiano
# Date: 2021-07-16 (July 16th, 2021)
#
"""
A class to standardize log formatting across all application artifacts.

Define common loggers and format to be used across the application.
NOTE: These logs are localized and non-persistent.
If used with a Docker container,
they cease to exist when the container does.

    Usage: (not meant to be called directly)
    log = create_logger("app_logging")
    log.debug("This message will be logged.")

"""
import logging


def create_logger(name):
    """This function creates a logger template for the discovery package.

    This funtion creates a consistant format and location for
    all application log files to write to.
    """
    print("Create logger with name %s" % name)
    logger = logging.getLogger(name)

    # It's okay to run INFO in Dev.  Turn it down to DEBUG for QA
    # and WARN for Prod unless troubleshooting an issue.
    logger.setLevel(logging.INFO)

    # create file handler which writes to a file.
    file_logger = logging.FileHandler("./discovery_output.log")
    file_logger.setLevel(logging.INFO)

    # create console handler with a higher log level
    console_logger = logging.StreamHandler()
    console_logger.setLevel(logging.INFO)

    # Create a custom formatter and add it to the handlers
    _format = "%(asctime)s - %(levelname)s - %(name)s - %(message)s"
    datefmt = "%m/%d/%Y %I:%M:%S %p"
    formatter = logging.Formatter(_format, datefmt)

    file_logger.setFormatter(formatter)
    console_logger.setFormatter(formatter)

    # Associate the the handlers to the loggers
    logger.addHandler(file_logger)
    logger.addHandler(console_logger)

    return logger


In [9]:
from dataclasses import dataclass


@dataclass
class Urn:
    """Dataclass which represents the base URN for the SML query.

    The base URN to be constructed as a string.

    Args:

    Attributes:
        specification: str
            The party ID specification.
        schema_id: str
            The party ID schema type.
        party_id: str
            The party ID
        urn: str
            The urn constructed by default values or passed into the
            dataclass when called.

    Returns:

    Raises:

    """

    specification: str
    schema_id: str
    party_id: str

    def urn(self) -> str:
        """Construct string for the party's URN"""
        # return str(f"{self.specification}:{self.schema}::{self.party_id}")
        return f"{self.specification}:{self.schema}::{self.party_id}"


In [10]:
import hashlib
import base64
from json import dumps
from dataclasses import dataclass


@dataclass
class Urn:

    specification: str
    schema: str
    party_id: str

    def urn(self) -> str:
        return f"{specification}:{schema}::{party_id}"

class Hasher:

    @staticmethod
    def hasher(specification, schema, party_id):
        urn = Urn(specification, schema, party_id)
        final_urn = urn.urn()
        final_urn_lower_case = final_urn.lower()
        urn_lower_encoded = final_urn_lower_case.encode("utf-8")
        urn_sha256_hashed = hashlib.sha256(urn_lower_encoded)
        urn_sha256_digest = urn_sha256_hashed.digest()
        urn_b32_hash = base64.b32encode(urn_sha256_digest)
        urn_b32_cleaned = urn_b32_hash.rstrip(b"=")
        lower_case_b32 = urn_b32_cleaned.lower()
        final_urn_b32 = lower_case_b32.decode("utf-8")
        return {
            "prty_id_spec": specification,
            "prty_id_schma_type": schema,
            "prty_id": party_id,
            "final_urn": final_urn_lower_case,
            "urn_hash": final_urn_b32,
        }

    def write_hashes_to_file(urn_dictionary, filename):
        json_str = dumps(urn_dictionary.__dict__)
        with open(filename, mode="w", encoding=str) as my_file:
            my_file.write(json_str)

            
            
record1 = {"spec": "urn:oasis:names:tc:ebcore:partyid-type:unregistered:myscheme", "schema": "BPC01", "party_id": "bpcBusid01"}
record2 = {"spec": "urn:oasis:names:tc:ebcore:partyid-type", "schema": "iso6523", "party_id": "0123456789"}
record3 = {"spec": "urn:oasis:names:tc:ebcore:partyid-type", "schema": "iso6523:0088", "party_id": "EAN-7638725972413"}
record4 = {"spec": "urn:oasis:names:tc:ebcore:partyid-type", "schema": "iso6523:0088", "party_id": "bpc-2343030383"}
record5 = {"spec": "urn:oasis:names:tc:ebcore:partyid-type", "schema": "iso6523:0088", "party_id": "4035811991021"}

my_list = [record1, record2, record3, record4, record5]
for record in my_list:
    hasher = Hasher()
    my_hash = hasher.hasher(record["spec"], record["schema"], record["party_id"])
    print(dumps(my_hash, indent=4))            
    
print(dumps(Hasher.hasher(record1["spec"], record1["schema"], record1["party_id"]), indent=4))    
print(dumps(Hasher.hasher(record2["spec"], record2["schema"], record2["party_id"]), indent=4))    
print(dumps(Hasher.hasher(record3["spec"], record3["schema"], record3["party_id"]), indent=4))
print(dumps(Hasher.hasher(record4["spec"], record4["schema"], record4["party_id"]), indent=4))
print(dumps(Hasher.hasher(record5["spec"], record5["schema"], record5["party_id"]), indent=4))

{
    "prty_id_spec": "urn:oasis:names:tc:ebcore:partyid-type:unregistered:myscheme",
    "prty_id_schma_type": "BPC01",
    "prty_id": "bpcBusid01",
    "final_urn": "urn:oasis:names:tc:ebcore:partyid-type:iso6523:0088::4035811991021",
    "urn_hash": "jc4swjyiphrll4gfhlu2edehpwlkmqmsncc2lc3so7m5jvgjkewa"
}
{
    "prty_id_spec": "urn:oasis:names:tc:ebcore:partyid-type",
    "prty_id_schma_type": "iso6523",
    "prty_id": "0123456789",
    "final_urn": "urn:oasis:names:tc:ebcore:partyid-type:iso6523:0088::4035811991021",
    "urn_hash": "jc4swjyiphrll4gfhlu2edehpwlkmqmsncc2lc3so7m5jvgjkewa"
}
{
    "prty_id_spec": "urn:oasis:names:tc:ebcore:partyid-type",
    "prty_id_schma_type": "iso6523:0088",
    "prty_id": "EAN-7638725972413",
    "final_urn": "urn:oasis:names:tc:ebcore:partyid-type:iso6523:0088::4035811991021",
    "urn_hash": "jc4swjyiphrll4gfhlu2edehpwlkmqmsncc2lc3so7m5jvgjkewa"
}
{
    "prty_id_spec": "urn:oasis:names:tc:ebcore:partyid-type",
    "prty_id_schma_type": "iso6523

In [11]:
record1 = {"spec": "urn:oasis:names:tc:ebcore:partyid-type:unregistered:myscheme", "schema": "BPC01", "party_id": "bpcBusid01"}
record2 = {"spec": "urn:oasis:names:tc:ebcore:partyid-type", "schema": "iso6523", "party_id": "0123456789"}
record3 = {"spec": "urn:oasis:names:tc:ebcore:partyid-type", "schema": "iso6523:0088", "party_id": "EAN-7638725972413"}
record4 = {"spec": "urn:oasis:names:tc:ebcore:partyid-type", "schema": "iso6523:0088", "party_id": "bpc-2343030383"}
record5 = {"spec": "urn:oasis:names:tc:ebcore:partyid-type", "schema": "iso6523:0088", "party_id": "4035811991021"}

my_list = [record1, record2, record3, record4, record5]
for record in my_list:
    hasher = Hasher()
    my_hash = hasher.hasher(record["spec"], record["schema"], record["party_id"])
    print(dumps(my_hash, indent=4))
    

{
    "prty_id_spec": "urn:oasis:names:tc:ebcore:partyid-type:unregistered:myscheme",
    "prty_id_schma_type": "BPC01",
    "prty_id": "bpcBusid01",
    "final_urn": "urn:oasis:names:tc:ebcore:partyid-type:iso6523:0088::4035811991021",
    "urn_hash": "jc4swjyiphrll4gfhlu2edehpwlkmqmsncc2lc3so7m5jvgjkewa"
}
{
    "prty_id_spec": "urn:oasis:names:tc:ebcore:partyid-type",
    "prty_id_schma_type": "iso6523",
    "prty_id": "0123456789",
    "final_urn": "urn:oasis:names:tc:ebcore:partyid-type:iso6523:0088::4035811991021",
    "urn_hash": "jc4swjyiphrll4gfhlu2edehpwlkmqmsncc2lc3so7m5jvgjkewa"
}
{
    "prty_id_spec": "urn:oasis:names:tc:ebcore:partyid-type",
    "prty_id_schma_type": "iso6523:0088",
    "prty_id": "EAN-7638725972413",
    "final_urn": "urn:oasis:names:tc:ebcore:partyid-type:iso6523:0088::4035811991021",
    "urn_hash": "jc4swjyiphrll4gfhlu2edehpwlkmqmsncc2lc3so7m5jvgjkewa"
}
{
    "prty_id_spec": "urn:oasis:names:tc:ebcore:partyid-type",
    "prty_id_schma_type": "iso6523

In [12]:
# Example #1
import hashlib
import base64

spec = "urn:oasis:names:tc:ebcore:partyid-type:unregistered:myscheme"
schema = "BPC01"
party_id = "bpcBusid01"

urn = spec + ":" + schema + "::" + party_id
print(urn)

urnl=urn.lower()
print(urnl)

urnle = urnl.encode('utf-8')
print(urnle)

s256 = hashlib.sha256(urnle)
print(s256)

s256d = s256.digest()
print(s256d)

b32 = base64.b32encode(s256d)
print(b32)

b32r = b32.rstrip(b"=")
print(b32r)

b32rd = b32r.decode('utf-8')
print(b32rd)

final = b32rd.lower()
print(final)

urn:oasis:names:tc:ebcore:partyid-type:unregistered:myscheme:BPC01::bpcBusid01
urn:oasis:names:tc:ebcore:partyid-type:unregistered:myscheme:bpc01::bpcbusid01
b'urn:oasis:names:tc:ebcore:partyid-type:unregistered:myscheme:bpc01::bpcbusid01'
<sha256 _hashlib.HASH object @ 0x10551b930>
b'\xc3{4\xfc3"\xdb\xc1u\xdcd\xe8\xbf\xe2\xad\x86\xdfjxob\x1e\'\x17\x8f\xb0\x83!\xec\x15\xab~'
b'YN5TJ7BTELN4C5O4MTUL7YVNQ3PWU6DPMIPCOF4PWCBSD3AVVN7A===='
b'YN5TJ7BTELN4C5O4MTUL7YVNQ3PWU6DPMIPCOF4PWCBSD3AVVN7A'
YN5TJ7BTELN4C5O4MTUL7YVNQ3PWU6DPMIPCOF4PWCBSD3AVVN7A
yn5tj7bteln4c5o4mtul7yvnq3pwu6dpmipcof4pwcbsd3avvn7a


In [13]:
# Example #2
import hashlib
import base64

spec = "urn:oasis:names:tc:ebcore:partyid-type"
schema = "iso6523"
party_id = "0123456789"

urn = spec + ":" + schema + "::" + party_id
print(urn)

urnl=urn.lower()
print(urnl)

urnle = urnl.encode('utf-8')
print(urnle)

s256 = hashlib.sha256(urnle)
print(s256)

s256d = s256.digest()
print(s256d)

b32 = base64.b32encode(s256d)
print(b32)

b32r = b32.rstrip(b"=")
print(b32r)

b32rd = b32r.decode('utf-8')
print(b32rd)

final = b32rd.lower()
print(final)

urn:oasis:names:tc:ebcore:partyid-type:iso6523::0123456789
urn:oasis:names:tc:ebcore:partyid-type:iso6523::0123456789
b'urn:oasis:names:tc:ebcore:partyid-type:iso6523::0123456789'
<sha256 _hashlib.HASH object @ 0x10551bdf0>
b'\xf0\xb5\xcaV\x0f\xbcK\x8b\x12\x08an\xc9S\x0c\xd8B\x9a\xb9\n\xa5\x98\xc4\x0e\x14\xce\x81\xac\xe7\x08P\xf0'
b'6C24UVQPXRFYWEQIMFXMSUYM3BBJVOIKUWMMIDQUZ2A2ZZYIKDYA===='
b'6C24UVQPXRFYWEQIMFXMSUYM3BBJVOIKUWMMIDQUZ2A2ZZYIKDYA'
6C24UVQPXRFYWEQIMFXMSUYM3BBJVOIKUWMMIDQUZ2A2ZZYIKDYA
6c24uvqpxrfyweqimfxmsuym3bbjvoikuwmmidquz2a2zzyikdya


In [14]:
# Example #3
import hashlib
import base64

spec = "urn:oasis:names:tc:ebcore:partyid-type"
schema = "iso6523:0088"
party_id = "EAN-7638725972413"

urn = spec + ":" + schema + "::" + party_id
print(urn)

urnl=urn.lower()
print(urnl)

urnle = urnl.encode('utf-8')
print(urnle)

s256 = hashlib.sha256(urnle)
print(s256)

s256d = s256.digest()
print(s256d)

b32 = base64.b32encode(s256d)
print(b32)

b32r = b32.rstrip(b"=")
print(b32r)

b32rd = b32r.decode('utf-8')
print(b32rd)

final = b32rd.lower()
print(final)

urn:oasis:names:tc:ebcore:partyid-type:iso6523:0088::EAN-7638725972413
urn:oasis:names:tc:ebcore:partyid-type:iso6523:0088::ean-7638725972413
b'urn:oasis:names:tc:ebcore:partyid-type:iso6523:0088::ean-7638725972413'
<sha256 _hashlib.HASH object @ 0x10551b250>
b'\x08\xb4\xd9\xaa\xf9\xe6\x80\xcb\xec\x1d\x8b\x12\x19\x00\x92\x7fh*\x0bh\x12\x1e\xe9({\x85\x9a\x9f\x07\x96I\xf5'
b'BC2NTKXZ42AMX3A5RMJBSAESP5UCUC3ICIPOSKD3QWNJ6B4WJH2Q===='
b'BC2NTKXZ42AMX3A5RMJBSAESP5UCUC3ICIPOSKD3QWNJ6B4WJH2Q'
BC2NTKXZ42AMX3A5RMJBSAESP5UCUC3ICIPOSKD3QWNJ6B4WJH2Q
bc2ntkxz42amx3a5rmjbsaesp5ucuc3iciposkd3qwnj6b4wjh2q


In [15]:
# Example #4
import hashlib
import base64

spec = "urn:oasis:names:tc:ebcore:partyid-type"
schema = "iso6523:0088"
party_id = "bpc-2343030383"

urn = spec + ":" + schema + "::" + party_id
print(urn)

urnl=urn.lower()
print(urnl)

urnle = urnl.encode('utf-8')
print(urnle)

s256 = hashlib.sha256(urnle)
print(s256)

s256d = s256.digest()
print(s256d)

b32 = base64.b32encode(s256d)
print(b32)

b32r = b32.rstrip(b"=")
print(b32r)

b32rd = b32r.decode('utf-8')
print(b32rd)

final = b32rd.lower()
print(final)

urn:oasis:names:tc:ebcore:partyid-type:iso6523:0088::bpc-2343030383
urn:oasis:names:tc:ebcore:partyid-type:iso6523:0088::bpc-2343030383
b'urn:oasis:names:tc:ebcore:partyid-type:iso6523:0088::bpc-2343030383'
<sha256 _hashlib.HASH object @ 0x10551ba30>
b'\x8a.\xa1\x87T\xed\xee_\xd6h!3\n\x81W\xect\x11m\x146I\x01\xef\xa8\t\x065\xb8\x80y|'
b'RIXKDB2U5XXF7VTIEEZQVAKX5R2BC3IUGZEQD35IBEDDLOEAPF6A===='
b'RIXKDB2U5XXF7VTIEEZQVAKX5R2BC3IUGZEQD35IBEDDLOEAPF6A'
RIXKDB2U5XXF7VTIEEZQVAKX5R2BC3IUGZEQD35IBEDDLOEAPF6A
rixkdb2u5xxf7vtieezqvakx5r2bc3iugzeqd35ibeddloeapf6a


In [16]:
# Example #5
import hashlib
import base64

spec = "urn:oasis:names:tc:ebcore:partyid-type"
schema = "iso6523:0088"
party_id = "4035811991021"

urn = spec + ":" + schema + "::" + party_id
print(urn)

urnl=urn.lower()
print(urnl)

urnle = urnl.encode('utf-8')
print(urnle)

s256 = hashlib.sha256(urnle)
print(s256)

s256d = s256.digest()
print(s256d)

b32 = base64.b32encode(s256d)
print(b32)

b32r = b32.rstrip(b"=")
print(b32r)

b32rd = b32r.decode('utf-8')
print(b32rd)

final = b32rd.lower()
print(final)

urn:oasis:names:tc:ebcore:partyid-type:iso6523:0088::4035811991021
urn:oasis:names:tc:ebcore:partyid-type:iso6523:0088::4035811991021
b'urn:oasis:names:tc:ebcore:partyid-type:iso6523:0088::4035811991021'
<sha256 _hashlib.HASH object @ 0x10551bef0>
b"H\xb9+'\x08y\xe2\xb5\xf0\xc5:\xe9\xa2\x0c\x87}\x96\xa6A\x92h\x85\xa5\x8brw\xd9\xd4\xd4\xc9Q,"
b'JC4SWJYIPHRLL4GFHLU2EDEHPWLKMQMSNCC2LC3SO7M5JVGJKEWA===='
b'JC4SWJYIPHRLL4GFHLU2EDEHPWLKMQMSNCC2LC3SO7M5JVGJKEWA'
JC4SWJYIPHRLL4GFHLU2EDEHPWLKMQMSNCC2LC3SO7M5JVGJKEWA
jc4swjyiphrll4gfhlu2edehpwlkmqmsncc2lc3so7m5jvgjkewa


In [17]:
import hashlib
import base64
from json import dumps
from dataclasses import dataclass


@dataclass
class Urn:

    specification: str
    schema: str
    party_id: str

    def urn(self) -> str:
        return f"{specification}:{schema}::{party_id}"

class Hasher:

    @staticmethod
    def hasher(specification, schema, party_id):
        urn = Urn(specification, schema, party_id)
        final_urn = urn.urn()
        final_urn_lower_case = final_urn.lower()
        urn_lower_encoded = final_urn_lower_case.encode("utf-8")
        urn_sha256_hashed = hashlib.sha256(urn_lower_encoded)
        urn_sha256_digest = urn_sha256_hashed.digest()
        urn_b32_hash = base64.b32encode(urn_sha256_digest)
        urn_b32_cleaned = urn_b32_hash.rstrip(b"=")
        lower_case_b32 = urn_b32_cleaned.lower()
        final_urn_b32 = lower_case_b32.decode("utf-8")
        return {
            "prty_id_spec": specification,
            "prty_id_schma_type": schema,
            "prty_id": party_id,
            "final_urn": final_urn_lower_case,
            "urn_hash": final_urn_b32,
        }

    def write_hashes_to_file(urn_dictionary, filename):
        json_str = dumps(urn_dictionary.__dict__)
        with open(filename, mode="w", encoding=str) as my_file:
            my_file.write(json_str)

            
            
record1 = {"spec": "urn:oasis:names:tc:ebcore:partyid-type:unregistered:myscheme", "schema": "BPC01", "party_id": "bpcBusid01"}
# record2 = {"spec": "urn:oasis:names:tc:ebcore:partyid-type", "schema": "iso6523", "party_id": "0123456789"}
# record3 = {"spec": "urn:oasis:names:tc:ebcore:partyid-type", "schema": "iso6523:0088", "party_id": "EAN-7638725972413"}
# record4 = {"spec": "urn:oasis:names:tc:ebcore:partyid-type", "schema": "iso6523:0088", "party_id": "bpc-2343030383"}
# record5 = {"spec": "urn:oasis:names:tc:ebcore:partyid-type", "schema": "iso6523:0088", "party_id": "4035811991021"}

print(dumps(Hasher.hasher(record1["spec"], record1["schema"], record1["party_id"]), indent=4))    
# print(dumps(Hasher.hasher(record2["spec"], record2["schema"], record2["party_id"]), indent=4))    
# print(dumps(Hasher.hasher(record3["spec"], record3["schema"], record3["party_id"]), indent=4))
# print(dumps(Hasher.hasher(record4["spec"], record4["schema"], record4["party_id"]), indent=4))
# print(dumps(Hasher.hasher(record5["spec"], record5["schema"], record5["party_id"]), indent=4))

{
    "prty_id_spec": "urn:oasis:names:tc:ebcore:partyid-type:unregistered:myscheme",
    "prty_id_schma_type": "BPC01",
    "prty_id": "bpcBusid01",
    "final_urn": "urn:oasis:names:tc:ebcore:partyid-type:iso6523:0088::4035811991021",
    "urn_hash": "jc4swjyiphrll4gfhlu2edehpwlkmqmsncc2lc3so7m5jvgjkewa"
}
